In [50]:

def  fnlpprint():
    import nlp.NLP
    
    
#     wav--------------------------------
    
    audio_path = r"nlp/test_audio/_67485707.wav"
    sentence = nlp.NLP.speech_to_text(audio_path)
    
    

    # m4a---------------------------
    
#     path = r"nlp/test_audio/_67559426.m4a"
#     path = r"nlp/test_audio/_67633155.m4a"    
#     path = r"nlp/test_audio/_67887224.m4a"
#     path = r"nlp/test_audio/_67903525.m4a"
    # path = r"nlp/test_audio/_67903526.m4a"
#     path = r"nlp/test_audio/_67903527.m4a" 
#     path = r"nlp/test_audio/_67903528.m4a"
#     path = r"nlp/test_audio/_67903530.m4a"   
#     path = r"nlp/test_audio/_67903531.m4a"
#     path = r"nlp/test_audio/_67903532.m4a" 
#     path = r"nlp/test_audio/_67903533.m4a"
#     path = r"nlp/test_audio/_67936273.m4a"
    

#     NLP.m4a_to_wav(path)  # 轉檔

#     audio_path = r'nlp/test_audio/speak.wav'
#     sentence = NLP.speech_to_text(audio_path)
    
    
    
    
    nlpprint=[]
    choose_product = int(0)
    if sentence==[]:
        nlpprint=("無此商品")
        choose_product=0
    else:
        seg_list = nlp.NLP.cut_word(sentence)  
        if seg_list==[]:
            nlpprint=("無此商品")
            choose_product=0
        else:
            product_name=nlp.NLP.get_category(seg_list)                # get_category
            # print("get_category後",product_name)  # 找出商品名稱

            if((product_name)==[]): 
                product_name=nlp.NLP.get_category2(seg_list)               # get_category2
                # print("get_category2後",product_name)

                if(product_name==[]): 
                    nlpprint=("無此商品")   
                    choose_product=0
            else:        
                if(len(product_name)==1):                     # 1筆商品        
                    choose_product=1
                    nlpprint=(product_name[0])  
                else:                                                   # n筆商品
                    # nlpprint.append("which product is your choose")
                    choose_product=2
                    for i in product_name:
                        nlpprint.append(i)
                        # print(i)    #--待寫---
                        
                    e='1'   #------------假設user選第二個----待寫swift input flask-----
                    e=int(e)
                    product_name[0]=product_name[e]
                    print(product_name[0])
        
    
    
    return nlpprint,choose_product



In [51]:
nlpprint,choose_product=(fnlpprint())
print(choose_product,nlpprint)

0 無此商品


In [53]:
def ocrprint(intn):               
    import ocr.OCR
    import os
    from google.cloud import vision
    import pymysql_func
    # key
    os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = "pubkey_group-five-project-4fbbbf03858d.json"

    YOUR_PIC = './uploads/{}'.format(intn)    
    texts = ocr.OCR.pic_to_text(YOUR_PIC)  # 圖片轉文字
    ocr_list=ocr.OCR.get_element(texts)

    disease= {"新陳代謝症候群":[], # disease= 疾病：預存成分
             "糖尿病":[],
             "高血脂":[],
             "高血壓":[],
             "腎臟病":[],
             "癌症":[],
             "word":[texts]}
    

    for key in  disease:         #將疾病對應之高風險添加物加入disease
            
        for e in ocr_list:
            outdisease=db.pymysql_func.find_disease(e)
            for i in outdisease:     
                
                if  i == key:
                    disease[key].append(e)
            # print(disease[key])
            
            
        if(bool(disease[key])==0):#無高風險添加物的疾病value為'無'
            disease[key]=['無']
                    
                    
                    
    disease['syndrome']=disease.pop('新陳代謝症候群')   #中文key改英文
    disease['diabeHyperlipidemiates']=disease.pop('糖尿病')
    disease['Hyperlipidemia']=disease.pop('高血脂')
    disease['hypertension']=disease.pop('高血壓')
    disease['kidney_disease']=disease.pop('腎臟病')
    disease['cancer']=disease.pop('癌症')
   
    
    
    
    disease_additives={'disease_additives':[{'syndrome':disease['syndrome']},   #改變格式
                                            {'diabeHyperlipidemiates':disease['diabeHyperlipidemiates']},
                                            {'Hyperlipidemia':disease['Hyperlipidemia']},
                                            {'hypertension':disease['hypertension']},
                                            {'kidney_disease':disease['kidney_disease']},
                                            {'cancer':disease['cancer']},
                                            {'word':disease['word']}]}
    

    # print('a')

    with open('./templates/tem_output.txt',mode='r+', encoding='utf-8') as f:                           #astar code        
        
        # print('aaa',disease_additives)  
        message =str(disease_additives)      #改寫&存取txt
            
            
        # print('b')
        # 先读取文件, 将文件指针指向开始，并使用truncate()清除所有内容
        data = f.read()
        # print(data)
        f.seek(0)
        f.truncate()
        f.write(message)
        # print(f.read())
        # print(message)
        f.close()
        # print('c')

    return disease_additives
# print(ocrprint('LINE_ALBUM_2022224_220225_22.png'))

In [54]:
# from flask import render_template 
import json 
from __future__ import unicode_literals
import os      # For File Manipulations like get paths, rename
from flask import Flask, flash, request, redirect, render_template, abort, Response, g, session, redirect, url_for
from werkzeug.utils import secure_filename

import tensorflow as tf
import keras


# app = Flask(__name__, static_url_path='/pathfinding',static_folder='./pathfinding')

app = Flask(__name__, static_url_path='/uploads',static_folder='./uploads')



app.secret_key = "secret key" # for encrypting the session
#It will allow below 16MB contents only, you can change it
app.config['MAX_CONTENT_LENGTH'] = 16 * 1024 * 1024 
path = os.getcwd()
# file Upload
UPLOAD_FOLDER = os.path.join(path, 'uploads')

if not os.path.isdir(UPLOAD_FOLDER):
    os.mkdir(UPLOAD_FOLDER)

app.config['UPLOAD_FOLDER'] = UPLOAD_FOLDER
ALLOWED_EXTENSIONS = set(['txt','png','docx','jpg'])

def allowed_file(filename):
    return '.' in filename and filename.rsplit('.', 1)[1].lower() in ALLOWED_EXTENSIONS




@app.route('/')
def upload_form():
    return render_template('upload.html')


# zeze's code
@app.route('/ocr/res', methods=['POST', 'GET'])
def show_res():
    with open('./templates/tem_output.txt',mode='r+', encoding='utf-8') as f:                           #astar code
        
        data = f.read()
        print(data)
        f.close()
    
    # a=render_template('helloworld.html')    
    # print(a)
    return json.dumps(eval(data),ensure_ascii=False),200,{'content-type':'application/json; charset=utf-8'}


@app.route('/ocr/img', methods=['POST', 'GET'])
def show_img():
    return render_template('show_image.html', data_result="http://34.72.27.77:5000/uploads/image.png")


        
        
        
        
        


@app.route('/',methods=["POST"])
def upload_file():
    # if request.method == 'POST':
        # check if the post request has the file part
    if 'file' not in request.files:
        flash('No file part')
        return redirect(request.url)
    file = request.files['file']
    if file.filename == '':
        flash('No file selected for uploading')
        return redirect(request.url)
    if file and allowed_file(file.filename):
        filename = secure_filename(file.filename)
        file.save(os.path.join(app.config['UPLOAD_FOLDER'], filename))
        outocr=ocrprint(file.filename)
        # print('d')
        # print('outocr',outocr)
        # return ' '
        # return redirect(url_for('abc', user_template=str(outocr)), code=307)


    
    
@app.route('/abc',methods = ['POST','GET'])
def abc():
    
    with open('./templates/helloworld.txt',mode='r+', encoding='utf-8') as f:                           #astar code
        
        data = f.read()
        print(data)
        f.close()
    
    # a=render_template('helloworld.html')    
    # print(a)
    return json.dumps(eval(data),ensure_ascii=False),200,{'content-type':'application/json; charset=utf-8'}




@app.route('/crawler/low/<instr>',methods=['GET'])
def crawler_low(instr):
    import crawler.price_low_high
    search=crawler.price_low_high.low_to_high(instr) 
    return json.dumps(search,ensure_ascii=False),200,{'content-type':'application/json; charset=utf-8'}


@app.route('/crawler/high/<instr>',methods=['GET'])
def crawler_high(instr):
    import crawler.price_high_low
    search=crawler.price_high_low.high_to_low(instr) 

    return json.dumps(search,ensure_ascii=False),200,{'content-type':'application/json; charset=utf-8'}


@app.route('/crawler/recommend/<instr>',methods=['GET'])

def crawler_recommend(instr):
    import crawler.crawler_product
    search=crawler.crawler_product.goods_info(instr) 

    return json.dumps(search,ensure_ascii=False),200,{'content-type':'application/json; charset=utf-8'}








# @app.route("/nlp_out",methods=['GET'])
# def testnlp_out():
#     nlpprint,choose_product=fnlpprint()


# # @app.route("/9/<intn>",methods=['GET'])
# # def hello_world3(intn):
# #     return str(intn)   


# @app.route("/input_png",methods=["GET","POST"])
# def input_png():
#     # username = request.args.get("username")
#     # userage = request.args.get("userage") 

#     data = request.get_json()                         #########
#     # data_str = data['image'] 





#     # return "Hello {},you are {} years old.".format(username,userage)
#     return str(data_str)

# @app.route("/10",methods=['GET'])
# def fnlp():
#     username = request.args.get("username")
#     userage = request.args.get("userage")    


#     # return "Hello {},you are {} years old.".format(username,userage)
#     return str(fnlpprint())


# @app.route("/hello_post",methods=["POST"]) 
# def hello_post():
#     outStr="""
#     <form action="/hello_post" method="POST">
#         <label>What's your name?</label>
#         <input name="username">
#         <button>SUBMIT</botton>
#     </form>
#     """
#     method=request.method #get or post
#     if method=="POST":
#         username=request.form.get("username")
#         outStr += """
#         <h1>Hello {}.<h1> 
#         """.format(username)


#     return outStr   




if __name__ == "__main__":
    app.run(host='0.0.0.0',port=5000,debug=False,threaded=True)

 * Serving Flask app '__main__' (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on all addresses.
 * Running on http://172.18.0.3:5000/ (Press CTRL+C to quit)
111.249.56.248 - - [22/Mar/2022 16:07:20] "GET / HTTP/1.1" 200 -
[2022-03-22 16:07:29,618] ERROR in app: Exception on / [POST]
Traceback (most recent call last):
  File "/opt/conda/lib/python3.9/site-packages/flask/app.py", line 2073, in wsgi_app
    response = self.full_dispatch_request()
  File "/opt/conda/lib/python3.9/site-packages/flask/app.py", line 1519, in full_dispatch_request
    return self.finalize_request(rv)
  File "/opt/conda/lib/python3.9/site-packages/flask/app.py", line 1538, in finalize_request
    response = self.make_response(rv)
  File "/opt/conda/lib/python3.9/site-packages/flask/app.py", line 1701, in make_response
    raise TypeError(
TypeError: The view function for 'upload_file' did not return a valid response. The function either returned None or ended without a return statement.
111.249.56.248 - - [22/Mar/2022 16:07:29] "POST / HTTP/1.1" 500 -


香糖-香甜葡萄口味原料:甜味劑木糖醇、麥芽糖醇、D-甘露醇、D-山梨醇、阿斯巴甜、蔗糖素、咀嚼基劑、麥芽糊精香料、大豆卵磷脂、檸檬酸、DL-蘋果酸反丁烯二酸、羧甲基纖維素鈉、澱粉、棕繼、阿拉伯膠、抗氧化劑二丁基羥基甲举。請勿吞食全素可食。苯酮尿症患香糖-香甜葡萄口味原料:甜味劑木糖醇、麥芽糖醇、D-甘露醇、D-山梨醇、阿斯巴甜、蔗糖素、咀嚼基劑、麥芽糊精香料、大豆卵磷脂、檸檬酸、DL-蘋果酸反丁烯二酸、羧甲基纖維素鈉、澱粉、棕繼、阿拉伯膠、抗氧化劑二丁基羥基甲举。請勿吞食全素可食。苯酮尿症患


111.249.56.248 - - [22/Mar/2022 16:07:30] "GET /abc HTTP/1.1" 200 -


{'disease_additives': [{'syndrome': ['無']}, {'diabeHyperlipidemiates': ['無']}, {'Hyperlipidemia': ['無']}, {'hypertension': ['無']}, {'kidney_disease': ['無']}, {'cancer': ['阿斯巴甜']}, {'word': ['香糖-香甜葡萄口味原料:甜味劑木糖醇、麥芽糖醇、D-甘露醇、D-山梨醇、阿斯巴甜、蔗糖素、咀嚼基劑、麥芽糊精香料、大豆卵磷脂、檸檬酸、DL-蘋果酸反丁烯二酸、羧甲基纖維素鈉、澱粉、棕繼、阿拉伯膠、抗氧化劑二丁基羥基甲举。請勿吞食全素可食。苯酮尿症患香糖-香甜葡萄口味原料:甜味劑木糖醇、麥芽糖醇、D-甘露醇、D-山梨醇、阿斯巴甜、蔗糖素、咀嚼基劑、麥芽糊精香料、大豆卵磷脂、檸檬酸、DL-蘋果酸反丁烯二酸、羧甲基纖維素鈉、澱粉、棕繼、阿拉伯膠、抗氧化劑二丁基羥基甲举。請勿吞食全素可食。苯酮尿症患']}]}


111.249.56.248 - - [22/Mar/2022 16:07:34] "GET /crawler/low/巧克力 HTTP/1.1" 200 -


價格由低到高排序後結果:


111.249.56.248 - - [22/Mar/2022 16:07:35] "GET /crawler/high/巧克力 HTTP/1.1" 200 -


價格由高到低排序後結果:


111.249.56.248 - - [22/Mar/2022 16:07:36] "GET /crawler/recommend/巧 HTTP/1.1" 200 -


In [56]:
# import db.pymysql_func

# a=db.pymysql_func.find_carrefour_goods('巧克力')
# b=db.pymysql_func.find_online_category('汽水')
# c=db.pymysql_func.find_disease('砂糖')
# d=db.pymysql_func.distinct_xy('巧克力')
# print('--------')
# print(a)
# print(b)
# print(c)
# print(d)


--------
[(67, 2, 3, 74, 41), (67, 2, 4, 74, 41), (67, 2, 5, 74, 41), (67, 2, 6, 74, 41), (67, 2, 7, 74, 45), (67, 2, 8, 74, 45), (67, 2, 9, 74, 45)]
[(51, 1, 10, 12), (51, 1, 3, 9), (51, 1, 5, 9)]
['新陳代謝症候群,糖尿病']
[(2, 74, 41), (2, 74, 45)]
